In [6]:
# ===== FILE SELECTION SECTION =====
# OPTION 1: Select individual files (just add/remove file paths)
selected_files = [
    r"C:\Users\Moses\math_ops\OperatorKernelO6\Core_Docs\Operator Centric Foundations of Godelian Incompleteness.md",
    r"C:\Users\Moses\math_ops\OperatorKernelO6\Core_Docs\Termination_Companion.md",
    # Add more files here - just paste the full path!
    r"C:\Users\Moses\math_ops\OperatorKernelO6\OperatorKernelO6\Meta\Termination.lean",
    # r"C:\Users\Moses\math_ops\OperatorKernelO6\Core_Docs\ordinal-toolkit.md",
]

# OPTION 2: Select whole folders (set to None if not using)
folder_to_convert = None  # Example: r"C:\Users\Moses\math_ops\OperatorKernelO6\Core_Docs"
file_pattern = "*.md"     # What files to include from folder: "*.md", "*.lean", "*.*"

# OUTPUT SETTINGS
output_dir = r"C:\Users\Moses\math_ops\OperatorKernelO6\Docs"
combined_title = "Custom_Selection"  # Title for combined document

print("📁 File Selection Configuration:")
print(f"   Individual files: {len(selected_files)} selected")
print(f"   Folder mode: {'ON' if folder_to_convert else 'OFF'}")
print(f"   Output: {output_dir}")
print("Ready to convert! Run the next cell.")

📁 File Selection Configuration:
   Individual files: 3 selected
   Folder mode: OFF
   Output: C:\Users\Moses\math_ops\OperatorKernelO6\Docs
Ready to convert! Run the next cell.


In [ ]:
# calll the script here

In [2]:
# c:\Users\Moses\math_ops\OperatorKernelO6\tools\build_pdfs.py
"""
Batch-converts every *.lean (meta/, kernel/) and *.md (core/docs/) file to
individual PDFs and then builds a combined PDF for each group.

REQUIREMENTS
------------
1. Python 3.10+
2. pandoc      – for Markdown → PDF
3. pdflatex    – for Lean   → PDF (uses `minted`, hence requires `-shell-escape`)
4. TinyTeX/MiKTeX/TeX Live with `minted` + `fvextra` packages installed
"""

import subprocess as sp
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Iterable

ROOT = Path(r"c:\Users\Moses\math_ops\OperatorKernelO6")
LEAN_DIRS  = [ROOT / "meta", ROOT / "kernel"]
MD_DIR     = ROOT / "core" / "docs"
OUT        = ROOT / "pdf"

def sh(cmd: list[str], cwd: Path | None = None) -> None:
    sp.run(cmd, cwd=cwd, check=True)

def tex_wrapper(code_file: Path) -> str:
    """Minimal LaTeX doc using minted."""
    return rf"""
\documentclass[11pt]{{article}}
\usepackage[margin=1in]{{geometry}}
\usepackage{{minted}}
\usemintedstyle{{borland}}
\begin{{document}}
\section*{{{code_file.name}}}
\begin{{minted}}[fontsize=\small]{{lean}}
\input{{{code_file.as_posix()}}}
\end{{minted}}
\end{{document}}
""".lstrip()

def convert_lean(fp: Path) -> Path:
    with TemporaryDirectory() as td:
        tex = Path(td) / f"{fp.stem}.tex"
        tex.write_text(tex_wrapper(fp), encoding="utf8")
        pdf = OUT / "lean" / f"{fp.stem}.pdf"
        pdf.parent.mkdir(parents=True, exist_ok=True)
        sh(["pdflatex", "-interaction=nonstopmode", "-shell-escape", tex.name], cwd=tex.parent)
        (tex.parent / f"{fp.stem}.pdf").replace(pdf)
    return pdf

def convert_md(fp: Path) -> Path:
    pdf = OUT / "docs" / f"{fp.stem}.pdf"
    pdf.parent.mkdir(parents=True, exist_ok=True)
    sh(["pandoc", "-f", "gfm", "-o", str(pdf), str(fp)])
    return pdf

def gather_files(dirs: Iterable[Path], suffix: str) -> list[Path]:
    return [p for d in dirs for p in d.rglob(f"*{suffix}")]

def build_combined(name: str, pieces: list[Path]) -> None:
    merged = OUT / f"{name}_combined.pdf"
    if not pieces:
        return
    sh(["pdftk", *map(str, pieces), "cat", "output", str(merged)])

def main() -> None:
    OUT.mkdir(exist_ok=True)
    # Lean
    lean_files = gather_files(LEAN_DIRS, ".lean")
    lean_pdfs  = [convert_lean(p) for p in lean_files]
    build_combined("lean", lean_pdfs)

    # Markdown
    md_files = gather_files([MD_DIR], ".md")
    md_pdfs  = [convert_md(p) for p in md_files]
    build_combined("docs", md_pdfs)

if __name__ == "__main__":
    main()

In [ ]:
# ===== CONVERSION SCRIPT =====
import os
from pathlib import Path
from datetime import datetime
import html

def create_html(title, content, file_path, file_type):
    """Create HTML from file content"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    if file_type == "markdown":
        # Basic markdown to HTML
        html_content = content
        html_content = html_content.replace('### ', '<h3>').replace('## ', '<h2>').replace('# ', '<h1>')
        html_content = html_content.replace('**', '<strong>').replace('*', '<em>')
        html_content = f'<div class="markdown">{html_content}</div>'
    else:
        # Lean code
        html_content = f'<pre class="lean-code">{html.escape(content)}</pre>'
    
    return f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>{title}</title>
    <style>
        body {{ 
            max-width: 1000px; margin: 0 auto; padding: 30px; 
            font-family: system-ui, sans-serif; line-height: 1.6; color: #333;
        }}
        h1 {{ color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }}
        h2 {{ color: #34495e; margin-top: 25px; }}
        h3 {{ color: #2c3e50; }}
        .lean-code {{ 
            background: #f8f9fa; padding: 20px; border-radius: 8px; 
            font-family: 'Consolas', monospace; font-size: 14px; 
            border: 1px solid #dee2e6; overflow-x: auto; white-space: pre-wrap;
        }}
        .file-info {{ 
            background: #e8f4fd; padding: 15px; border-radius: 8px; 
            margin-bottom: 20px; font-size: 0.9em;
        }}
        .markdown {{ line-height: 1.7; }}
    </style>
</head>
<body>
    <h1>{title}</h1>
    <div class="file-info">
        <strong>File:</strong> {file_path}<br>
        <strong>Type:</strong> {file_type.title()}<br>
        <strong>Generated:</strong> {timestamp}
    </div>
    {html_content}
</body>
</html>
    """.strip()

# Determine which files to process
files_to_process = []

if folder_to_convert and Path(folder_to_convert).exists():
    # Option 2: Folder mode
    folder_path = Path(folder_to_convert)
    found_files = list(folder_path.glob(file_pattern))
    files_to_process = [str(f) for f in found_files]
    print(f"📂 Folder mode: Found {len(files_to_process)} files in {folder_to_convert}")
else:
    # Option 1: Individual files
    files_to_process = [f for f in selected_files if Path(f).exists()]
    missing = [f for f in selected_files if not Path(f).exists()]
    if missing:
        print(f"⚠️  Missing files: {len(missing)}")
        for f in missing:
            print(f"   ❌ {Path(f).name}")

print(f"\\n🚀 Processing {len(files_to_process)} files...")

# Create output directory
Path(output_dir).mkdir(exist_ok=True)

# Process each file
processed_files = []
for i, file_path in enumerate(files_to_process, 1):
    file_path = Path(file_path)
    file_name = file_path.stem
    file_type = "markdown" if file_path.suffix == ".md" else "lean"
    
    print(f"[{i}/{len(files_to_process)}] {file_name}")
    
    # Read content
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
        if not content.strip():
            content = "-- File appears to be empty --"
    except Exception as e:
        print(f"   ❌ Error reading file: {e}")
        continue
    
    # Create HTML
    html_content = create_html(file_name, content, str(file_path), file_type)
    
    # Save individual HTML file
    safe_name = "".join(c for c in file_name if c.isalnum() or c in '-_')
    output_path = Path(output_dir) / f"{safe_name}.html"
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(html_content)
    
    print(f"   ✅ Created: {safe_name}.html")
    
    processed_files.append({
        'name': file_name,
        'content': content,
        'type': file_type,
        'path': str(file_path)
    })

# Create combined document if multiple files
if len(processed_files) > 1:
    print(f"\\n📚 Creating combined document...")
    
    combined_content = f"# {combined_title}\\n\\nCombined document with {len(processed_files)} files.\\n\\n"
    combined_content += "## Table of Contents\\n\\n"
    
    for file in processed_files:
        combined_content += f"- [{file['name']}]({file['path']})\\n"
    
    combined_content += "\\n---\\n\\n"
    
    for file in processed_files:
        combined_content += f"## {file['name']}\\n\\n"
        combined_content += f"**Source:** `{file['path']}`\\n\\n"
        combined_content += f"```{file['type']}\\n{file['content']}\\n```\\n\\n---\\n\\n"
    
    # Create combined HTML
    combined_html = create_html(f"{combined_title} Combined", combined_content, "Combined Document", "markdown")
    combined_path = Path(output_dir) / f"{combined_title}_Combined.html"
    
    with open(combined_path, 'w', encoding='utf-8') as f:
        f.write(combined_html)
    
    print(f"   ✅ Created: {combined_title}_Combined.html")

print(f"\\n🎉 CONVERSION COMPLETE!")
print(f"📂 Output directory: {output_dir}")
print(f"📄 Files processed: {len(processed_files)}")
if len(processed_files) > 1:
    print(f"📚 Combined document: {combined_title}_Combined.html")

print("\\n💡 To customize: Edit the first cell and run both cells again!")